# Importing Library


In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.regularizers as regularizers
import csv
import pathlib

In [2]:
import tensorboard
import datetime
import os
tensorboard.__version__

'2.3.0a20200713'

In [7]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22622 sha256=dbb683d9dc66fa42363dbd66a57f6b3ff593fcb681d9ee580cfee495578f9bfc
  Stored in directory: c:\users\rishi johri\appdata\local\pip\cache\wheels\b4\c3\39\9c01ae2b4726f37024bba5592bec868b47a2fab5a786e8979a
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8374 sha256=d178f1c3db5e9a25ddc6009b0f6824e5149a805cf582fd5a99c8f1cf01cfa827
  Stored in directory: c:\users\rishi johri\appdata\local\pip\cache\wheels\57\6d\a3\a39b839cc75274d2acfb1c58bfead2f726c6577fe8c4723f13
Successfully built yfinance multitasking


In [5]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
import yfinance as yf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import pandas as pd

In [9]:
# torch and sklearn imports
import torch
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader


# Parameters for ML Models / Creating Functions

## parameters for Models

In [ ]:
window_length = 10
batch = 32
valid_split = 0.2
epochs = 500
learn_rate = 0.001
pred_length = 12
logdir="logs/fit/plot_"+str(window_length)+"_"+str(pred_length)
checkpoint_path = "training_"+str(window_length)+"_"+str(pred_length)+"/cp.ckpt"


company = "AAPL"
start = "2004-08-19" #yyyy/mm/dd ( for testing 2004-09-01 to 2004-09-17)
end = "2021-01-17"

## Functions for Model

In [ ]:
# callback for Tensorboard Graph
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)
es_callback =  tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=6, min_delta=0.01)
def normal(prev, num):
    return (num-prev)/prev

def row_maker(datas, key):
    outer = []
    for data in datas:
        for column in data:
            outer.append(normal(data[column].iloc[key-1], data[column].iloc[key]))
        for i in range(0, 3):
            if i==0:
                outer.append(data["Close"].iloc[key]-data["Close"].iloc[key-1])
            else:
                outer.append(data["Close"].iloc[key]-data["Close"].iloc[key-i*5])
    return outer
def row_master(data, key, ema):
    outer = []
    for column in data:
        outer.append(normal(data[column].iloc[key-1], data[column].iloc[key]))
    for q3 in range(0, 3):
        if q3==0:
            outer.append(data["Close"].iloc[key]-data["Close"].iloc[key-1])
        else:
            outer.append(data["Close"].iloc[key]-data["Close"].iloc[key-q3*5])
    for q1 in range(1, 4):
        sum = 0
        for i in range(0, q1*5):
            sum += data["Close"].iloc[key-i]
        outer.append(sum/(q1*5*data["Close"].iloc[key]))
    curr = data["Close"].iloc[key]
    multiplier = 1.5/(10 + 1)
    ema_curr =  normal(data["Close"].iloc[key-1], data["Close"].iloc[key])*multiplier + ema*(1-multiplier)
    outer.append(ema_curr)
    return outer, ema_curr

## random set parameters

In [ ]:
import random 
def Rand(start, end, num): 
    res = [] 
    for j in range(num): 
        res.append(random.randint(start, end)) 
    return res 
# Driver Code 
num = 500
start = 0
end = 4000
random_list=Rand(start, end, num)
print(random_list) 

[2299, 2076, 1322, 416, 2290, 3949, 2398, 3638, 2633, 333, 909, 3925, 3314, 1021, 2415, 1345, 2758, 739, 324, 878, 2796, 913, 1193, 2998, 1264, 100, 3861, 1884, 2644, 2912, 104, 1699, 2981, 3920, 2457, 2826, 1466, 1560, 1972, 3485, 1948, 2303, 95, 3672, 2951, 80, 2193, 3133, 3531, 757, 266, 1895, 230, 1069, 2088, 3000, 3419, 3961, 2390, 3116, 322, 238, 625, 2737, 3663, 2619, 1153, 1224, 1439, 2710, 685, 3137, 1000, 1046, 3468, 669, 706, 1148, 3521, 3252, 3441, 3274, 357, 2216, 3907, 1307, 3395, 3152, 3860, 2638, 2307, 2359, 833, 1970, 109, 3063, 3882, 3336, 1417, 2901, 731, 1924, 3778, 308, 2348, 3899, 1946, 837, 2410, 937, 1103, 1911, 2508, 3277, 2274, 3104, 271, 937, 982, 3180, 2851, 3467, 2266, 3103, 2314, 2747, 1420, 3751, 3958, 1631, 3433, 582, 3244, 3357, 3022, 2702, 537, 2906, 1711, 2999, 1940, 1827, 2838, 1449, 1052, 1357, 3470, 3389, 2015, 2206, 541, 3175, 678, 3356, 2405, 1361, 5, 413, 3957, 39, 2031, 1839, 2411, 3174, 3571, 3326, 1112, 400, 3064, 2144, 2746, 1183, 3723, 32, 

# Reading data from Files

## for EOD

### INTC_Final

In [ ]:
# for INTC_Final
heading_EOD_INTC = "Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume"
data_INTC = pd.read_csv("INTC_final.csv", names=heading_EOD_INTC.split(","))
dates = data_INTC.pop("Date")
data_INTC.pop("Split")
data_INTC.pop("Dividend")
copy_INTC = data_INTC.copy()
y_INTC = copy_INTC["Close"]
pd.read_csv("INTC_final.csv", names=heading_EOD_INTC.split(","))

### INTC

In [ ]:
# for EOD_INTC
heading_EOD_INTC = "Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume" 
data_INTC = pd.read_csv("EOD-INTC.csv", names=heading_EOD_INTC.split(","))
dates = data_INTC.pop("Date")
data_INTC.pop("Split")
data_INTC.pop("Dividend")
copy_INTC = data_INTC.copy()
y_INTC = copy_INTC["Close"]
pd.read_csv("EOD-INTC.csv", names=heading_EOD_INTC.split(","))

### AAPL

In [ ]:
# for EOD_AAPL
heading_EOD_AAPL = "Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume"
heading_EOD_AAPL_m = "Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,Momentum1,Momentum3,Momentum5,Adj_Momentum1,Adj_Momentum3,Adj_Momentum5"
data_AAPL = pd.read_csv("EOD-AAPL_m.csv", names=heading_EOD_AAPL_m.split(","))
dates = data_AAPL.pop("Date")
data_AAPL.pop("Split")
data_AAPL.pop("Dividend")
copy_AAPL = data_AAPL.copy()
y_AAPL = copy_AAPL["Open"]
pd.read_csv("EOD-AAPL_m.csv", names=heading_EOD_AAPL_m.split(","))

## for combined Dataset from Websites

In [ ]:
# 3 for MSFT, 9 for AAPL, 15 for GOOG
company = "aapl"
start = "2004-08-19" #yyyy/mm/dd ( for testing 2004-09-01 to 2004-09-17)
end = "2021-01-17"
msft = yf.download("MSFT", start=start, end=end)
goog = yf.download("GOOGL", start=start, end=end)
aapl = yf.download("AAPL", start=start, end=end)
converter = {
    "aapl":aapl,
    "msft":msft,
    "goog":goog,

}
data = []
y = []
y2 = []
for i in range(9, msft.shape[0]-1):
    data.append(row_maker([aapl, msft, goog], i))
    y.append(normal(converter[company]["Close"].iloc[i-1], converter[company]["Close"].iloc[i]))
data = np.array(data)
y = np.array(y)
print(data)
print(data.shape)
print(y)
y_tmp = np.zeros((y.shape[0], 2))
for j in range(0, y.shape[0]-1):
    if y[j]<y[j+1]: 
        y_tmp[j][1] = 1 # rise case
    else:
        y_tmp[j][0] = 1 # fall case
y = y_tmp
tensor_y_COMB = tf.convert_to_tensor(y)
tensor_x_COMB = tf.convert_to_tensor(data)
ones = 0
zeros = 0
for ping in range(0, tensor_y_COMB.shape[0]):
    if tensor_y_COMB[ping][1]==1:
        ones = ones+1
    else:
        zeros = zeros+1
print("in train set -> ones are : {} and zeros are : {}".format(ones/(ones+zeros), zeros/(ones+zeros)))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[[ 6.75086395e-03  2.97568089e-02  5.58826495e-03 ... -1.06106186e+00
  -2.87788010e+00 -1.67744482e+03]
 [ 3.49854443e-02 -5.00141110e-03  1.87188632e-02 ...  6.30630493e-01
  -3.20320511e+00  5.85586548e-01]
 [-1.38028502e-02  3.07183810e-03  5.16798119e-03 ... -7.50751495e-01
  -3.07307434e+00 -4.15415573e+00]
 ...
 [-5.34098946e-03 -3.68745283e-03 -1.27626412e-02 ... -1.88599854e+01
  -2.61999512e+00 -3.65299072e+01]
 [ 2.02330356e-03  1.35707801e-02  1.28488481e-02 ...  9.81994629e+00
   2.43699951e+01 -1.05100098e+01]
 [ 1.58434967e-02 -3.42333175e-03  2.10124525e-03 ... -1.63299561e+01
  -4.34199219e+01 -5.32995605e+00]]
(4122, 27)
[ 0.0397217  -0.00557728 -0.01205832 ... -0.00139551  0.01622668
 -0.01512717]
in train set -> ones are : 0.48423095584667636 and zeros are : 0.51

## for Individual Company from Websites

In [ ]:
comp = yf.download(company, start=start, end=end)
comp = comp.dropna()
data = []
y = []
ema_curr = 0;

for i in range(0, 15):
    ema_curr = comp["Close"].iloc[i]
ema_curr = ema_curr/(15*comp["Close"].iloc[14])
for i in range(15, comp.shape[0]-1):
    row, ema_curr = row_master(comp, i, ema_curr)
    data.append(row)
    y.append(normal(comp["Close"].iloc[i-1], comp["Close"].iloc[i]))
data = np.array(data)
y = np.array(y)
y_tmp = np.zeros((y.shape[0], 2))
for j in range(0, y.shape[0]-1):
    if y[j]<y[j+1]: 
        y_tmp[j][1] = 1 # rise case
    else:
        y_tmp[j][0] = 1 # fall case
y = y_tmp
print(y[5])
print(data[5][:])
tensor_y_COMB = tf.convert_to_tensor(y)
tensor_x_COMB = tf.convert_to_tensor(data)
ones = 0
zeros = 0
for ping in range(0, tensor_y_COMB.shape[0]):
    if tensor_y_COMB[ping][1]==1:
        ones = ones+1
    else:
        zeros = zeros+1
print("in train set -> ones are : {} and zeros are : {}".format(ones/(ones+zeros), zeros/(ones+zeros)))

[*********************100%***********************]  1 of 1 completed
[1. 0.]
[3.83522009e-02 1.68661816e-02 3.76282847e-02 2.17330959e-02
 2.17330399e-02 7.81005936e-04 1.41071081e-02 2.26785541e-02
 2.64285803e-02 9.68066815e-01 9.65751245e-01 9.57027485e-01
 3.31157440e-02]
in train set -> ones are : 0.4842079689018465 and zeros are : 0.5157920310981535


## For Excel Created Combined Dataset

In [ ]:
# for combined dataset
company = "AAPL"
heading_final = "Date,OM,HM,LM,CM,ACM,VM,OA,HA,LA,CA,ACA,VA,OG,HG,LG,CG,ACG,VG,MA1,MA5,MA10,MG1,MG5,MG10,MM1,MM5,MM10,ON,HN,LN,CN,ACN"
data_AAPL = pd.read_csv("final.csv", names=heading_final.split(","))
# data_AAPL = data_AAPL.drop(0)
data_AAPL.dropna()
# print(data_AAPL)
dates = data_AAPL.pop("Date")
data_AAPL.pop("ON")
data_AAPL.pop("HN")
data_AAPL.pop("LN")
data_AAPL.pop("CN")
data_AAPL.pop("ACN")

# data_AAPL = pd.concat([data_AAPL.pop(x) for x in ["Open","High","Low","Close","Adj Close","Volume","MA1","MA5","MA10"]], axis=1)
copy_AAPL = data_AAPL.copy()
if (company=="AAPL"):
    y_AAPL = copy_AAPL.pop("CA")
elif (company=="GOOG"):
    y_AAPL = copy_AAPL.pop("CG")
elif (company=="MSFT"):
    y_AAPL = copy_AAPL.pop("CM")
else:
    company = "AAPL"
    y_AAPL = copy_AAPL.pop("CA")
data_AAPL = data_AAPL.to_numpy()
print(data_AAPL)

[[-2.1986440e-03  7.3206440e-03  3.3271720e-03 ...  9.0000000e-02
  -1.6000000e-01  5.0000000e-01]
 [ 6.2431140e-03  5.8139530e-03  7.7376940e-03 ...  2.3000200e-01
   1.8000000e-01  5.0000000e-01]
 [ 2.1897450e-03 -2.1675940e-03 -9.1407680e-03 ... -5.1000000e-01
  -3.4999800e-01 -9.0000000e-02]
 ...
 [-1.1454947e-02 -1.5661490e-03  2.8594880e-03 ...  1.4100030e+00
   4.0899960e+00 -7.8099980e+00]
 [ 8.8309500e-03  3.2294340e-03 -5.5625110e-03 ... -3.3199920e+00
  -5.2699890e+00 -8.6599890e+00]
 [-1.1069427e-02 -1.3565768e-02 -3.3374350e-03 ... -3.7001000e-01
  -6.9700010e+00 -9.7700040e+00]]


In [ ]:
data_AAPL==data

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
# for DIS_final.csv
heading_DIS = "Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,3-Month,1-Yr,10Yr,SMA,EMA,Returns,Momentum1,Momdiv5,Momentum5,Ret,RSI,Term Spread,SVAR"
data_DIS = pd.read_csv("DIS_final.csv", names=heading_DIS.split(","))
data_DIS.pop("Date")
data_DIS.pop("SMA")
data_DIS.pop("EMA")
data_DIS.pop("SVAR")
data_DIS.pop("Term Spread")
# data_DIS.pop("Adj_Open")
# data_DIS.pop("Adj_Close")
# data_DIS.pop("Adj_High")
# data_DIS.pop("Adj_Low")
# data_DIS.pop("Adj_Volume")
copy_DIS = data_DIS.copy()
y_DIS = copy_DIS["Close"]
pd.read_csv("DIS_final.csv", names=heading_DIS.split(","))

In [ ]:
# for INTC_final
heading_INTC = "Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,pb,pe1,sps,bvps,gp,divyield,3-Month,1-Yr,10Yr,SMA,EMA,Returns,Momentum1,Momdiv5,Momentum5,Ret,RSI,Term Spread,Yield_gap,SVAR"
data_INTC = pd.read_csv("INTC_final.csv", names=heading_INTC.split(","))
data_INTC.pop("Date")
copy_INTC = data_INTC.copy()
y_INTC = copy_INTC["Close"]
pd.read_csv("INTC_final.csv", names=heading_INTC.split(","))

In [ ]:
# for AAPL_final
heading_AAPL = "Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,3-Month,1-Yr,10-Yr,SMA,EMA,Returns,Momentum1,Momdiv5,Momentum5,Ret,RSI,Term Spread,SVAR" 
pd.read_csv("AAPL_final.csv", names=heading_AAPL.split(","))
data_AAPL = pd.read_csv("AAPL_final.csv", names=heading_AAPL.split(","))
dates = data_AAPL.pop("Date")
# data_AAPL.pop("SMA")
# data_AAPL.pop("EMA")
# data_AAPL.pop("SVAR")
# data_AAPL.pop("Term Spread")
# data_AAPL.pop("Dividend")
# data_AAPL.pop("Split")
# data_AAPL.pop("3-Month")
# data_AAPL.pop("1-Yr")
# data_AAPL.pop("10-Yr")
# data_AAPL.pop("Returns")
# data_AAPL.pop("Momentum1")
# data_AAPL.pop("Momdiv5")
# data_AAPL.pop("Adj_Volume")
copy_AAPL = data_AAPL.copy()
y_AAPL = copy_AAPL["Close"]
pd.read_csv("AAPL_final.csv", names=heading_AAPL.split(","))

,Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,3-Month,1-Yr,10-Yr,SMA,EMA,Returns,Momentum1,Momdiv5,Momentum5,Ret,RSI,Term Spread,SVAR
0,12/28/2017,-0.102377,-0.105786,-0.097326,-0.102538,-0.914318,-0.079869,0,2.251448,2.243003,2.276761,2.253864,-1.152898,2.761231,2.812532,0.582766,2.380214,2.366710,-0.041877,0.044393,-0.734083,-1.049027,0.083831,-0.051558,-1.753956,-0.187720
1,12/27/2017,-0.108161,-0.112607,-0.102306,-0.105619,-0.686712,-0.079869,0,2.217804,2.203126,2.247822,2.235911,-0.963687,2.890398,2.789488,0.553143,2.384703,2.374389,-0.042207,0.019953,-0.847458,-1.231235,0.000718,-0.156520,-1.753956,-0.184723
2,12/26/2017,-0.103662,-0.108208,-0.102507,-0.105811,-0.156602,-0.079869,0,2.243972,2.228841,2.246657,2.234789,-0.523002,2.967898,2.789488,0.701256,2.388853,2.386053,-0.015236,-0.222819,-1.161784,-1.750663,-0.804225,-0.370436,-1.668295,-0.184715
3,12/22/2017,-0.078730,-0.083002,-0.071327,-0.077320,-0.920248,-0.079869,0,2.389015,2.376201,2.427841,2.400849,-1.157828,2.606231,2.743400,0.730878,2.384137,2.399687,-0.042208,0.018324,0.000326,0.123086,-0.004824,0.349439,-1.616898,-0.190153
4,12/21/2017,-0.082007,-0.079202,-0.073914,-0.077320,-0.711583,-0.079869,0,2.369950,2.398413,2.412808,2.400849,-0.984362,2.657898,2.743400,0.730878,2.361501,2.389484,-0.041608,0.054169,0.303853,0.599002,0.114455,0.659279,-1.616898,-0.188886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,10/21/2013,2.087364,2.141097,2.108351,2.145222,-1.016921,-0.079869,0,-1.621842,-1.588700,-1.598951,-1.577113,1.978472,-0.726268,-0.989732,1.175215,-1.688060,-1.692522,-0.017037,0.695697,1.338597,0.747012,0.767423,1.201074,1.415516,-0.145169
929,10/18/2013,2.050223,2.045217,2.070775,2.065189,-1.191167,-0.079869,0,-1.650293,-1.662506,-1.627702,-1.638535,0.964506,-0.726268,-0.966688,1.086348,-1.704653,-1.711443,-0.039035,0.256751,0.990314,0.529651,0.269363,0.503943,1.346986,-0.133023
930,10/17/2013,2.011603,2.016657,2.031776,2.037018,-1.251021,-0.079869,0,-1.679876,-1.684490,-1.657542,-1.660154,0.616207,-0.700435,-0.943644,1.115970,-1.720619,-1.723578,-0.040295,0.202222,0.915861,0.481667,0.208085,0.290048,1.346986,-0.129090
931,10/16/2013,2.016808,2.002313,2.028865,2.015290,-1.255058,-0.079869,0,-1.675888,-1.695532,-1.659769,-1.676829,0.592716,-0.571268,-0.897556,1.352950,-1.731424,-1.734208,-0.041210,0.150518,0.636387,0.318808,0.148971,0.786872,1.449780,-0.125197


# Output Data Processing

## Output data processing AAPL

In [ ]:
# output data processing AAPL
print(y_AAPL)
y =[]

# For Binary Loss
# for i in range(0, y_AAPL.shape[0]-1):
#   if y_AAPL[i]<y_AAPL[i+1]:
#     y_tmp_AAPL[] = 1
#   else:
#     y_tmp_AAPL[i] = 0

# for i in range(0, y_AAPL.shape[0]-1):
#     y_tmp_AAPL[i] = y_AAPL[i+1]
# y_tmp_AAPL[y_AAPL.shape[0]-1] = y_tmp_AAPL[y_AAPL.shape[0]-2]

# 1 is for rise and 0 is for fall
for i in range(0, y_AAPL.shape[0]-1):
    # row, ema_curr = row_master(comp, i, ema_curr)
    # data.append(row)
    y.append(normal(y_AAPL[i], y_AAPL[i+1]))
# data = np.array(data)
y.append(normal(y_AAPL[y_AAPL.shape[0]-2], y_AAPL[y_AAPL.shape[0]-1]))
y = np.array(y)


# for i in range(0, y_AAPL.shape[0]-1):
#   if y_AAPL[i] < y_AAPL[i+1]:
#     y_tmp_AAPL[i][0] = 0
#     y_tmp_AAPL[i][1] = 1
#   else:
#     y_tmp_AAPL[i][0] = 1
#     y_tmp_AAPL[i][1] = 0

y_AAPL = y
print(y_AAPL)
# ping = 0
# for bill in y_AAPL==tensor_y_COMB:
#     if bill[0]==False:
#         ping = ping + 1
# print(ping)

0       0.039721
1      -0.005577
2      -0.012059
3       0.015044
4       0.016499
          ...   
4118   -0.023249
4119   -0.001396
4120    0.016227
4121   -0.015127
4122   -0.013731
Name: CA, Length: 4123, dtype: float64
[-1.14039302  1.16243692 -2.24749573 ... -1.93224066 -0.09232528
 -0.09232528]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


## output data processing INTC

In [ ]:
print(y_INTC)
y_tmp_INTC = np.zeros((y_INTC.shape[0], 2))

for i in range(0, y_INTC.shape[0]-1):
  if y_INTC[i]<y_INTC[i+1]:
    y_tmp_INTC[i][0] = 1
  else:
    y_tmp_INTC[i][0] = 0

  

for i in range(0, y_INTC.shape[0]-pred_length):
  if y_INTC[i]<y_INTC[i+pred_length]:
    y_tmp_INTC[i][1] = 1
  else:
    y_tmp_INTC[i][1] = 0

y_INTC = y_tmp_INTC
print(y_INTC)



## output data processing DIS

In [ ]:
y_tmp_DIS = np.zeros((y_DIS.shape[0], 2))

for i in range(1, y_DIS.shape[0]):
  if y_DIS[i-1]<y_DIS[i]:
    y_tmp_DIS[i-1][0] = 1
    y_tmp_DIS[i-1][1] = 0
  else:
    y_tmp_DIS[i-1][0] = 0
    y_tmp_DIS[i-1][1] = 1

y_tmp_DIS[y_DIS.shape[0]-1][0] = 0
y_tmp_DIS[y_DIS.shape[0]-1][0] = 0
y_DIS = y_tmp_DIS


# Tensorflow Data Processing

## tensorflow data processing for DIS

In [ ]:
tensor_x_DIS = tf.convert_to_tensor(data_DIS)
tensor_y_DIS = tf.convert_to_tensor(y_DIS)

## tensorflow data processing for AAPL

In [ ]:
tensor_x_COMB = tf.convert_to_tensor(data_AAPL)
tensor_y_COMB = tf.convert_to_tensor(y_AAPL)

## tensorflow data processing for INTC

In [ ]:
tensor_x_INTC = tf.convert_to_tensor(data_INTC)
tensor_y_INTC = tf.convert_to_tensor(y_INTC)

## TSNE conversions

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')
out_m.write("OM\tHM\tLM\tCM\tACM\tVM\tOpen\tHigh\tLow\tClose\tAdj Close\tVolume\tOG\tHG\tLG\tCG\tACG\tVG\tMA1\tMA5\tMA10\tDate\n")
for i in range(0, tensor_x_AAPL.shape[0]):
    for j in range(0, tensor_x_AAPL.shape[1]):
        out_m.write(str(tensor_x_AAPL[i][j])+"\t")
    out_m.write(str(dates[i])+"\n")
    out_v.write(str(tensor_y_AAPL[i][0])+"\t"+str(tensor_y_AAPL[i][1])+"\n")

# Time Series Data Processing for LSTM

## Time Series Data processing for DIS

In [ ]:
window_x_DIS = np.zeros((tensor_x_DIS.shape[0]-window_length, window_length, tensor_x_DIS.shape[1]))
window_y_DIS = np.zeros((tensor_y_DIS.shape[0]-window_length, 2))

for i in range(0, tensor_x_DIS.shape[0]-window_length):
  window_x_DIS[i] = tensor_x_DIS[i:i+window_length][:]
  window_y_DIS[i] = tensor_y_DIS[i+window_length-1][:]

indices = tf.range(start=0, limit=window_x_DIS.shape[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

window_x_DIS = tf.gather(window_x_DIS, shuffled_indices)
window_y_DIS = tf.gather(window_y_DIS, shuffled_indices)


train_window_x_DIS, test_window_x_DIS = tf.split(window_x_DIS, [window_x_DIS.shape[0]-int(window_x_DIS.shape[0]*(valid_split)), int(window_x_DIS.shape[0]*(valid_split))])
train_window_y_DIS, test_window_y_DIS = tf.split(window_y_DIS, [window_y_DIS.shape[0]-int(window_y_DIS.shape[0]*(valid_split)), int(window_y_DIS.shape[0]*(valid_split))])
print(test_window_x_DIS.shape)
print(train_window_x_DIS.shape)

## time series data processing for random AAPL

In [ ]:
window_x_AAPL = np.zeros((len(random_list), window_length, tensor_x_AAPL.shape[1]))
window_y_AAPL = np.zeros((len(random_list), 2))

for p in enumerate(random_list):
  i=p[1]
  window_x_AAPL[p[0]] = tensor_x_AAPL[i:i+window_length][:]
  window_y_AAPL[p[0]] = tensor_y_AAPL[i+window_length-1][:]

indices = tf.range(start=0, limit=window_x_AAPL.shape[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

window_x_AAPL = tf.gather(window_x_AAPL, shuffled_indices)
window_y_AAPL = tf.gather(window_y_AAPL, shuffled_indices)

train_window_x_AAPL, test_window_x_AAPL = tf.split(window_x_AAPL, [window_x_AAPL.shape[0]-int(window_x_AAPL.shape[0]*(valid_split)), int(window_x_AAPL.shape[0]*(valid_split))])
train_window_y_AAPL, test_window_y_AAPL = tf.split(window_y_AAPL, [window_y_AAPL.shape[0]-int(window_y_AAPL.shape[0]*(valid_split)), int(window_y_AAPL.shape[0]*(valid_split))])
print(test_window_x_AAPL.shape)
print(train_window_y_AAPL.shape)

(100, 10, 32)
(400, 2)


## time series data processing for AAPL

In [ ]:
window_x_AAPL = np.zeros((tensor_x_AAPL.shape[0]-window_length, window_length, tensor_x_AAPL.shape[1]))
window_y_AAPL = np.zeros((tensor_y_AAPL.shape[0]-window_length, 2))

for i in range(0, tensor_x_AAPL.shape[0]-window_length):
  window_x_AAPL[i] = tensor_x_AAPL[i:i+window_length][:]
  window_y_AAPL[i] = tensor_y_AAPL[i+window_length-1][:]

indices = tf.range(start=0, limit=window_x_AAPL.shape[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

window_x_AAPL = tf.gather(window_x_AAPL, shuffled_indices)
window_y_AAPL = tf.gather(window_y_AAPL, shuffled_indices)

train_window_x_AAPL, test_window_x_AAPL = tf.split(window_x_AAPL, [window_x_AAPL.shape[0]-int(window_x_AAPL.shape[0]*(valid_split)), int(window_x_AAPL.shape[0]*(valid_split))])
train_window_y_AAPL, test_window_y_AAPL = tf.split(window_y_AAPL, [window_y_AAPL.shape[0]-int(window_y_AAPL.shape[0]*(valid_split)), int(window_y_AAPL.shape[0]*(valid_split))])
print(test_window_x_AAPL.shape)
print(train_window_x_AAPL.shape)

(822, 10, 32)
(3291, 10, 32)


## time series data processing for INTC

In [ ]:
# time series data processing for INTC

window_x_INTC = np.zeros((tensor_x_INTC.shape[0]-window_length, window_length, tensor_x_INTC.shape[1]))
window_y_INTC = np.zeros((tensor_y_INTC.shape[0]-window_length,2))

for i in range(0, tensor_x_INTC.shape[0]-window_length):
  window_x_INTC[i] = tensor_x_INTC[i:i+window_length][:]
  window_y_INTC[i] = tensor_y_INTC[i+window_length-1][:]

indices = tf.range(start=0, limit=window_x_INTC.shape[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

window_x_INTC = tf.gather(window_x_INTC, shuffled_indices)
window_y_INTC = tf.gather(window_y_INTC, shuffled_indices)

train_window_x_INTC, test_window_x_INTC = tf.split(window_x_INTC, [window_x_INTC.shape[0]-int(window_x_INTC.shape[0]*(valid_split)), int(window_x_INTC.shape[0]*(valid_split))])
train_window_y_INTC, test_window_y_INTC = tf.split(window_y_INTC, [window_y_INTC.shape[0]-int(window_y_INTC.shape[0]*(valid_split)), int(window_y_INTC.shape[0]*(valid_split))])
print(test_window_x_INTC.shape)
print(train_window_x_INTC.shape)

## time series data processing for Combined

In [ ]:
# time series processing for combined
window_x_COMB = np.zeros((tensor_x_COMB.shape[0]-window_length, window_length, tensor_x_COMB.shape[1]))
window_y_COMB = np.zeros((tensor_y_COMB.shape[0]-window_length, 2))

for i in range(0, tensor_x_COMB.shape[0]-window_length):
  window_x_COMB[i] = tensor_x_COMB[i:i+window_length][:]
  window_y_COMB[i] = tensor_y_COMB[i+window_length-1][:]

indices = tf.range(start=0, limit=window_x_COMB.shape[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

window_x_COMB = tf.gather(window_x_COMB, shuffled_indices)
window_y_COMB = tf.gather(window_y_COMB, shuffled_indices)

train_window_x_COMB, test_window_x_COMB = tf.split(window_x_COMB, [window_x_COMB.shape[0]-int(window_x_COMB.shape[0]*(valid_split)), int(window_x_COMB.shape[0]*(valid_split))])
train_window_y_COMB, test_window_y_COMB = tf.split(window_y_COMB, [window_y_COMB.shape[0]-int(window_y_COMB.shape[0]*(valid_split)), int(window_y_COMB.shape[0]*(valid_split))])
print(test_window_x_COMB.shape)
print(train_window_x_COMB.shape)

(822, 10, 27)
(3291, 10, 27)


# Dataset Conversion for Model

In [ ]:
# x_train_final = tf.concat([train_window_x_AAPL, train_window_x_DIS], axis=0)
# y_train_final = tf.concat([train_window_y_AAPL, train_window_y_DIS], axis=0)
# x_test_final = tf.concat([test_window_x_AAPL, test_window_x_DIS], axis=0)
# y_test_final = tf.concat([test_window_y_AAPL, test_window_y_DIS], axis=0)

# x_train_final = train_window_x_AAPL
# y_train_final = train_window_y_AAPL
# x_test_final = test_window_x_AAPL
# y_test_final = test_window_y_AAPL

x_train_final = train_window_x_COMB
y_train_final = train_window_y_COMB
x_test_final = test_window_x_COMB
y_test_final = test_window_y_COMB

# x_train_final = train_window_x_DIS
# y_train_final = train_window_y_DIS
# x_test_final = test_window_x_DIS
# y_test_final = test_window_y_DIS

# x_train_final = train_window_x_INTC
# y_train_final = train_window_y_INTC
# x_test_final = test_window_x_INTC
# y_test_final = test_window_y_INTC

ones = 0
zeros = 0
for ping in range(0, y_train_final.shape[0]):
    if y_train_final[ping][1]==1:
        ones = ones+1
    else:
        zeros = zeros+1
print("in train set -> ones are : {} and zeros are : {}".format(ones/(ones+zeros), zeros/(ones+zeros)))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_final, y_train_final))
train_dataset = train_dataset.shuffle(buffer_size=x_train_final.shape[0])
train_dataset = train_dataset.batch(batch, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test_final, y_test_final))
test_dataset = test_dataset.shuffle(buffer_size=x_test_final.shape[0])
test_dataset = test_dataset.batch(batch, drop_remainder=True)

in train set -> ones are : 0.4925554542692191 and zeros are : 0.5074445457307809


# Models

## Simple Neural Network (Classification) Creation and Compilation

In [ ]:
def NN_model():
    X_input = tf.keras.Input((tensor_x_COMB.shape[1]))
    X = X_input
    # X = layers.Flatten()(X)
    X = layers.LayerNormalization()(X)
    X = layers.Dense(30, activation="relu")(X)
    X_res = X
    X = layers.Dense(35, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(40, activation="relu")(X)
    X = layers.Dense(45, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(50, activation="relu")(X)
    X = layers.Dense(30, activation="relu")(X)
    X = layers.ReLU()(tf.add(X_res, X))
    X = layers.LayerNormalization()(X)
    X = layers.Dense(25, activation="relu", kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(20, activation="relu")(X)
    X = layers.Dense(15, activation="relu")(X)
    X = layers.Dense(25, activation="relu", kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(10, activation="relu")(X)
    X = layers.Dense(50, activation="relu")(X)
    X = layers.Dense(25, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(10, activation="relu")(X)
    X = layers.Dense(2, activation="softmax")(X)
    return tf.keras.Model(inputs=X_input, outputs=X)

model = NN_model()
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

## Simple Neural Network (Regressive) Creation and Compilation

In [ ]:
def NN_model():
    X_input = tf.keras.Input((tensor_x_COMB.shape[1]))
    X = X_input
    # X = layers.Flatten()(X)
    X = layers.LayerNormalization()(X)
    X = layers.Dense(30, activation="relu")(X)
    X_res = X
    X = layers.Dense(35, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(40, activation="relu")(X)
    X = layers.Dense(45, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(50, activation="relu")(X)
    X = layers.Dense(30, activation="relu")(X)
    X = layers.ReLU()(tf.add(X_res, X))
    X = layers.LayerNormalization()(X)
    X = layers.Dense(25, activation="relu", kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(20, activation="relu")(X)
    X = layers.Dense(15, activation="relu")(X)
    X = layers.Dense(25, activation="relu", kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(10, activation="relu")(X)
    X = layers.Dense(50, activation="relu")(X)
    X = layers.Dense(25, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(10, activation="relu")(X)
    X = layers.Dense(1, activation="tanh")(X)
    return tf.keras.Model(inputs=X_input, outputs=X)

model = NN_model()
model.compile(
    optimizer="adam",
    loss='mean_absolute_error',
    metrics=["mean_absolute_error"]
)

## Windowed Neural Network Creation and Compilation

In [ ]:
def NN_model_windowed():
    X_input = tf.keras.Input((window_length, tensor_x_COMB.shape[1]), batch_size=batch)
    X = X_input
    X = layers.Flatten()(X)
    X = layers.LayerNormalization()(X)
    X = layers.Dense(30*window_length, activation="relu")(X)
    X_res = X
    X = layers.Dense(35*window_length, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(40*window_length, activation="relu")(X)
    X = layers.Dense(45*window_length, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(50*window_length, activation="relu")(X)
    X = layers.Dense(30*window_length, activation="relu")(X)
    X = layers.ReLU()(tf.add(X_res, X))
    X = layers.LayerNormalization()(X)
    X = layers.Dense(25*window_length, activation="relu", kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(20*window_length, activation="relu")(X)
    X = layers.Dense(15*window_length, activation="relu")(X)
    X = layers.Dense(25*window_length, activation="relu", kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(10*window_length, activation="relu")(X)
    X = layers.Dense(50*window_length, activation="relu")(X)
    X = layers.Dense(25*window_length, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(10*window_length, activation="relu")(X)
    X = layers.Dense(2*window_length, activation="relu")(X)
    X = layers.Dense(10, activation="relu")(X)
    X = layers.Dense(7, activation="relu")(X)
    X = layers.Dense(4, activation="relu")(X)
    X = layers.Dense(2, activation="softmax")(X)
    return tf.keras.Model(inputs=X_input, outputs=X)

model = NN_model_windowed()
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

## simple time series Model creation and Compilation

In [ ]:
# simple LSTM creation and compilation
 
def time_series_model():
    X_input = tf.keras.Input((window_length, x_train_final.shape[2]), batch_size=batch) 
    X = X_input
    X = layers.LayerNormalization()(X)
    X = layers.Dense(30, activation="relu")(X)
    X_res = X
    X = layers.Dense(35, activation="relu",)(X)
    X = layers.Dense(40, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(45, activation="relu",)(X)
    X = layers.Dense(50, activation="relu",)(X)
    X = layers.Dense(30, activation="relu",)(X)
    X = layers.ReLU()(tf.add(X_res, X))
    X = layers.LayerNormalization()(X)
    X = layers.Dropout(0.2)(X)
    X = layers.Dense(25, activation="relu", kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(20, activation="relu")(X)
    X = layers.Dense(15, activation="relu")(X)
    X = layers.Dense(25, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01) )(X)
    X = layers.Dense(10, activation="relu")(X)
    X = layers.Dense(50, activation="relu")(X)
    X = layers.Dense(25, activation="relu",)(X)
    X = layers.Dropout(0.2)(X)
    X = layers.Dense(10, activation="relu")(X)
    X = layers.Flatten()(X)
    X = layers.Dense(50, activation="relu")(X)
    X = layers.Dense(30, activation="relu",kernel_regularizer=regularizers.L1L2(0.01, 0.01))(X)
    X = layers.Dense(5, activation="relu")(X)
    X = layers.Dense(2, activation="softmax",)(X)
    
    return tf.keras.Model(inputs=X_input, outputs=X)
 
model = time_series_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learn_rate, ),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)  


In [ ]:
# Convolution NN creation and compile

# Summary

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 27)]         0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 27)           54          input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           840         layer_normalization[0][0]        
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 35)           1085        dense[0][0]                      
______________________________________________________________________________________________

# Model Fitting

## Weight Loading

In [ ]:
model.load_weights(checkpoint_path)

## Fitting for Time Series

In [ ]:
model.fit(train_dataset,
          validation_data=test_dataset,
          epochs=200,
          callbacks=[cp_callback, 
                    tensorboard_callback,
                    es_callback
                     ]
          )

## Fitting for Neural Network

In [ ]:
# for NN Model
model.fit(x=tensor_x_COMB, 
          y=tensor_y_COMB, 
          validation_split=0.2, 
          batch_size=batch,
          epochs=200,
          shuffle=True,
          callbacks=[cp_callback, 
                     tensorboard_callback, 
                     es_callback
                     ]
          )

Epoch 1/200
103/103 [==============================] - 9s 12ms/step - loss: 8.4770 - accuracy: 0.5650 - val_loss: 4.5905 - val_accuracy: 0.6772

Epoch 00001: saving model to training_10_12/cp.ckpt
Epoch 2/200
103/103 [==============================] - 1s 6ms/step - loss: 3.7302 - accuracy: 0.6908 - val_loss: 1.8204 - val_accuracy: 0.7306

Epoch 00002: saving model to training_10_12/cp.ckpt
Epoch 3/200
103/103 [==============================] - 1s 6ms/step - loss: 1.4816 - accuracy: 0.7057 - val_loss: 0.8564 - val_accuracy: 0.7379

Epoch 00003: saving model to training_10_12/cp.ckpt
Epoch 4/200
103/103 [==============================] - 1s 6ms/step - loss: 0.8045 - accuracy: 0.7228 - val_loss: 0.7031 - val_accuracy: 0.7294

Epoch 00004: saving model to training_10_12/cp.ckpt
Epoch 5/200
103/103 [==============================] - 1s 7ms/step - loss: 0.7013 - accuracy: 0.7145 - val_loss: 0.6749 - val_accuracy: 0.7367

Epoch 00005: saving model to training_10_12/cp.ckpt
Epoch 6/200
103/103

# Self Testing

### For Combined Data Processing

In [ ]:
# get historical market data
num_days = 60
hist_msft = yf.download("MSFT", period="{}d".format(num_days))
hist_goog = yf.download("GOOGL", period="{}d".format(num_days))
hist_aapl = yf.download("AAPL", period="{}d".format(num_days))
print(str(hist_aapl.tail(num_days-10)))
right = 0;
part_right = 0;
mega_right = 0;
wrong = 0
part_wrong =0
for key in range(10, num_days-1):
    print("Stocks of company {} on date : {} ==>".format(company, hist_aapl.iloc[key].name))
    c2 = hist_aapl["Close"].iloc[key]
    c1 = hist_aapl["Close"].iloc[key-1]
    c3 = hist_aapl["Close"].iloc[key+1]
    give = [row_maker([hist_msft, hist_aapl, hist_goog], key)]
    probability = model.predict(give)[0]
    prob = probability[1]
    riser = False
    uncertain = False
    if prob<0.5:
        riser = False
        if (c2<c1):
            print(" fall {}".format(prob))
            uncertain = False
        else: 
            uncertain = True
            print(" uncertain {} but less than {}".format(prob, c2*c2/c1))
    elif prob>0.5:
        riser = True
        if (c2>c1):
            uncertain = False
            print(" rise {}".format(prob))
        else:
            uncertain = True
            print(" uncertain {} but greater than than {}".format(prob, c2*c2/c1))
    else :
        print(" uncertain {}".format(prob))
    if uncertain:
        if riser and c3>c2*c2/c1:
            if c3<c2:
                print("     partially right on rise")
                part_right +=1
            else:
                print("     maga right on rise")
                mega_right += 1
        elif not riser and c3<c2*c2/c1:
            if c3>c2:
                print("     partially right on fall")
                part_right +=1
            else:
                print("     mega right on fall")
                mega_right += 1
        else:
            print("     partially wrong")
            part_wrong+=1
    else:
        if riser:
            if (c3>c2):
                print("     right prediction on rise")
                right +=1
            else:
                print("     wrong prediction on rise")
                wrong+=1
        else :
            if (c3<c2):
                print("     right prediction on fall")
                right+=1
            else:
                wrong+=1
                print("     wrong prediction on fall")
key = num_days-1
print("--------UNSEEN FUTURE-----------")
print("Stocks of company {} on date : {} ==>".format(company, hist_aapl.iloc[key].name))
c2 = hist_aapl["Close"].iloc[key]
c1 = hist_aapl["Close"].iloc[key-1]
give = [row_maker([hist_msft, hist_aapl, hist_goog], key)]
probability = model.predict(give)[0]
if probability[1]<0.5:
    if (c2<c1):
        print(" fall {}".format(probability[1]))
    else: 
        print(" uncertain {} but less than {}".format(probability[1], c2*c2/c1))
elif probability[1]>0.5:
    if (c2>c1):
        print(" rise {}".format(probability[1]))
    else:
        print(" uncertain {} but greater than than {}".format(probability[1], c2*c2/c1))
else :
    print(" uncertain {}".format(probability[1]))
tot = right+mega_right+part_right+part_wrong+wrong
print("right answers {},mega right ans {}, part right ans {}, part wrong {}, wrong ans {}".format(right/tot,mega_right/tot, part_right/tot, part_wrong/tot, wrong/tot))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                  Open        High  ...   Adj Close     Volume
Date                                ...                       
2020-11-13  119.440002  119.669998  ...  119.260002   81581900
2020-11-16  118.919998  120.989998  ...  120.300003   91183000
2020-11-17  119.550003  120.669998  ...  119.389999   74271000
2020-11-18  118.610001  119.820000  ...  118.029999   76322100
2020-11-19  117.589996  119.059998  ...  118.639999   74113000
2020-11-20  118.639999  118.769997  ...  117.339996   73604300
2020-11-23  117.180000  117.620003  ...  113.849998  127959300
2020-11-24  113.910004  115.849998  ...  115.169998  113874200
2020-11-25  115.550003  116.750000  ...  116.029999   76499200
2020-11-27  116.570000  117.489998  ...  116.589996   46691300
2020-11-30  116.970001  120.970001  .

### for Single Company Classification Based

In [ ]:
num_days = 60
comp = yf.download(company, period="{}d".format(num_days))
# for i in range(10, num_days):
#     print("date {} and Close {}".format(comp.iloc[i].name, comp["Close"].iloc[i]))

right = 0;
part_right = 0;
mega_right = 0;
wrong = 0
part_wrong =0
ema_curr = 0
benefit_range = 0
waste_range = 0
for i in range(0, 15):
    ema_curr = comp["Close"].iloc[i]
ema_curr = ema_curr/(15*comp["Close"].iloc[14])
for key in range(15, num_days-1):
    try:
        print("Stocks of company {} with close {} from date : {} ==>".format(company,comp["Close"].iloc[key], comp.iloc[key].name))
        c2 = comp["Close"].iloc[key]
        c1 = comp["Close"].iloc[key-1]
        c3 = comp["Close"].iloc[key+1]
        row, ema_curr = row_master(comp, key, ema_curr)
        give = [row]
        probability = model.predict(give)[0]
        prob = probability[1]
        riser = False
        uncertain = False
        if prob<0.5:
            riser = False
            if (c2<c1):
                print("     fall {}".format(prob))
                uncertain = False
            else: 
                uncertain = True
                
                print("     uncertain {} but less than {}".format(prob, c2*c2/c1))
        elif prob>0.5:
            riser = True
            if (c2>c1):
                uncertain = False
                print("     rise {}".format(prob))
            else:
                uncertain = True
                
                print("     uncertain {} but greater than than {}".format(prob, c2*c2/c1))
        else :
            print("     uncertain {}".format(prob))
        if uncertain:
            if riser and c3>c2*c2/c1:
                if c3<c2:
                    print("         partially right on rise")
                    part_right +=1
                else:
                    print("         maga right on rise")
                    mega_right += 1
                if (comp["High"].iloc[key+1]>c2*c2/c1):
                    benefit_range += 1
                else:
                    waste_range += 1
            elif not riser and c3<c2*c2/c1:
                if c3>c2:
                    print("         partially right on fall")
                    part_right +=1
                else:
                    print("         mega right on fall")
                    mega_right += 1
                if (comp["Low"].iloc[key+1]<c2*c2/c1):
                    benefit_range += 1
                else:
                    waste_range += 1
            else:
                print("         partially wrong")
                part_wrong+=1
        else:
            if riser:
                if (c3>c2):
                    print("         right prediction on rise")
                    right +=1
                else:
                    print("         wrong prediction on rise")
                    wrong+=1
            else :
                if (c3<c2):
                    print("         right prediction on fall")
                    right+=1
                else:
                    wrong+=1
                    print("         wrong prediction on fall")
    except:
        print("         error encountered")
key = num_days-1
print("-----------UNSEEN FUTURE--------------")
try:
    print("Stocks of company {} on date : {} >>==>".format(company, comp.iloc[key].name))
    c2 = comp["Close"].iloc[key]
    c1 = comp["Close"].iloc[key-1]
    row, ema_curr = row_master(comp, key, ema_curr)
    give = [row]
    probability = model.predict(give)[0]
    if probability[1]<0.5:
        if (c2<c1):
            print("     fall {}".format(probability[1]))
        else: 
            print("     uncertain {} but less than {}".format(probability[1], c2*c2/c1))
    elif probability[1]>0.5:
        if (c2>c1):
            print("     rise {}".format(probability[1]))
        else:
            print("     uncertain {} but greater than than {}".format(probability[1], c2*c2/c1))
    else :
        print("     uncertain {}".format(probability[1]))
except:
    print("     error in future")
tot = right+mega_right+part_right+part_wrong+wrong

print("right answers {}".format(right/tot))
print("mega right ans {}".format(mega_right/tot))
print("part right ans {}".format(part_right/tot))
print("part wrong {}".format(part_wrong/tot))
print("wrong ans {}".format(wrong/tot))
print("benefitial range {},\nnon-beneficial range {}".format(benefit_range/(benefit_range+waste_range), waste_range/(benefit_range+waste_range)))

[*********************100%***********************]  1 of 1 completed
Stocks of company AAPL with close 113.8499984741211 from date : 2020-11-23 00:00:00 ==>
     uncertain 0.8151711225509644 but greater than than 110.46380225914352
         maga right on rise
Stocks of company AAPL with close 115.16999816894531 from date : 2020-11-24 00:00:00 ==>
     uncertain 0.18536174297332764 but less than 116.5053022047242
         partially right on fall
Stocks of company AAPL with close 116.02999877929688 from date : 2020-11-25 00:00:00 ==>
     uncertain 0.47781506180763245 but less than 116.89642121010137
         partially right on fall
Stocks of company AAPL with close 116.58999633789062 from date : 2020-11-27 00:00:00 ==>
     uncertain 0.48450255393981934 but less than 117.15269662223574
         partially wrong
Stocks of company AAPL with close 119.05000305175781 from date : 2020-11-30 00:00:00 ==>
     uncertain 0.08477733284235 but less than 121.56191501669588
         partially wrong


## for Regressive Outputs

In [ ]:
num_days = 1000
comp = yf.download(company, period="{}d".format(num_days))

right = 0;
part_right = 0;
mega_right = 0;
wrong = 0
part_wrong =0
ema_curr = 0
benefit_range = 0
waste_range = 0
perc = 0
exp_perc = 0
true_perc = 0
exp_bool = 0
for i in range(0, 15):
    ema_curr = comp["Close"].iloc[i]
ema_curr = ema_curr/(15*comp["Close"].iloc[14])
for key in range(15, num_days-1):
    print("Stocks of company {} with close {} from date : {} ==>".format(company,comp["Close"].iloc[key], comp.iloc[key].name))
    c2 = comp["Close"].iloc[key]
    c1 = comp["Close"].iloc[key-1]
    c3 = comp["Close"].iloc[key+1]
    row, ema_curr = row_master(comp, key, ema_curr)
    give = [row]
    probability = model.predict(give)[0]
    prob = probability[0]
    riser = False
    uncertain = False
    expect = (prob + 1)*c2
    percent_err = abs((c3-expect)*100/c3)
    perc += percent_err
    exp_perc += abs(prob)
    true_perc += abs((c3-c2)/c2)
    e1 = expect>c2
    e2 = c3>c2
    
    print("ans  {} -- {}".format(e1, e2))
    if e1==e2:
        exp_bool += 1
    print(" expected {} true is {} with deviation of {} and percent_err error of {}".format(expect, c3, c3-expect, percent_err))
    # print(" expected percentage {} true percentage is {} and dev is {}".format(prob, abs(c3-c2)/c2))
perc = perc/(num_days-16)
exp_perc = exp_perc/(num_days-16)
true_perc = true_perc/(num_days-16)
print("exp bool is {}".format(exp_bool))
print("avg precentage is {}".format(perc))
print("market percent is {}, our percent is {}".format(true_perc, exp_perc))
key = num_days-1
print("-----------UNSEEN FUTURE--------------")
try:
    print("Stocks of company {} on date : {} ==>".format(company, comp.iloc[key].name))
    c2 = comp["Close"].iloc[key]
    c1 = comp["Close"].iloc[key-1]
    row, ema_curr = row_master(comp, key, ema_curr)
    give = [row]
    probability = model.predict(give)[0]
    print("expected is {}".format(probability))
    
except:
    print("error in future")
tot = right+mega_right+part_right+part_wrong+wrong
# print("right answers {}".format(right/tot))
# print("mega right ans {}".format(mega_right/tot))
# print("part right ans {}".format(part_right/tot))
# print("part wrong {}".format(part_wrong/tot))
# print("wrong ans {}".format(wrong/tot))
# print("benefitial range {},\n non-beneficial_range {}".format(benefit_range/(benefit_range+waste_range), waste_range/(benefit_range+waste_range)))

[*********************100%***********************]  1 of 1 completed
Stocks of company DIS with close 110.08999633789062 from date : 2017-02-28 00:00:00 ==>
ans  True -- True
 expected 110.24425558972425 true is 111.04000091552734 with deviation of 0.7957453258030966 and percent_err error of 0.7166294301532405
Stocks of company DIS with close 111.04000091552734 from date : 2017-03-01 00:00:00 ==>
ans  True -- False
 expected 111.19559132368997 true is 110.58999633789062 with deviation of -0.6055949857993426 and percent_err error of 0.5476037669347965
Stocks of company DIS with close 110.58999633789062 from date : 2017-03-02 00:00:00 ==>
ans  True -- True
 expected 110.74495619494255 true is 111.23999786376953 with deviation of 0.4950416688269854 and percent_err error of 0.445021285808761
Stocks of company DIS with close 111.23999786376953 from date : 2017-03-03 00:00:00 ==>
ans  True -- False
 expected 111.39586850974332 true is 110.66999816894531 with deviation of -0.7258703407980107 

# Graphs

In [ ]:
rm -rf ./logs/

In [ ]:
%tensorboard --logdir logs

In [ ]:
!zip -r /content/log.zip /content/logs
from google.colab import files
files.download("/content/log.zip")

In [ ]:
!unzip log.zip

# Miscelleneous Code

In [ ]:
# # time series processing for DIS dataset LSTM

# window_tensor_x_DIS = np.zeros((tensor_x_DIS.shape[0]-window_length, window_length, tensor_x_DIS.shape[1]))
# window_tensor_y_DIS = np.zeros((tensor_y_DIS.shape[0]-window_length))
# train_window_x_DIS = np.zeros((tensor_x_DIS.shape[0]-window_length, window_length, tensor_x_DIS.shape[1]))
# test_window_y_DIS = np.zeros((tensor_y_DIS.shape[0]-window_length))
# x_train_DIS, x_test_DIS = tf.split(tensor_x_DIS, [tensor_x_DIS.shape[0]-int(tensor_x_DIS.shape[0]*(valid_split)), int(tensor_x_DIS.shape[0]*(valid_split))])
# y_train_DIS, y_test_DIS = tf.split(tensor_y_DIS, [tensor_y_DIS.shape[0]-int(tensor_y_DIS.shape[0]*(valid_split)), int(tensor_y_DIS.shape[0]*(valid_split))])

# test_window_x_DIS = np.zeros((x_test_DIS.shape[0]-window_length, window_length, tensor_x_DIS.shape[1]))
# test_window_y_DIS = np.zeros((y_test_DIS.shape[0]-window_length))

# train_window_x_DIS = np.zeros((x_train_DIS.shape[0]-window_length, window_length, x_train_DIS.shape[1]))
# train_window_y_DIS = np.zeros((y_train_DIS.shape[0]-window_length))

# for i in range(0, x_train_DIS.shape[0]-window_length):
#   train_window_x_DIS[i] = x_train_DIS[i:i+window_length][:]
#   train_window_y_DIS[i] = y_train_DIS[i+window_length-1]

# # train_dataset_DIS = tf.data.Dataset.from_tensor_slices((train_window_x_DIS, train_window_y_DIS))

# test_window_x_DIS = np.zeros((x_test_DIS.shape[0]-window_length, window_length, x_test_DIS.shape[1]))
# test_window_y_DIS = np.zeros((y_test_DIS.shape[0]-window_length))
# for i in range(0, x_test_DIS.shape[0]-window_length):
#   test_window_x_DIS[i] = x_test_DIS[i:i+window_length][:]
#   test_window_y_DIS[i] = y_test_DIS[i+window_length-1]

# # test_dataset_DIS = tf.data.Dataset.from_tensor_slices((test_window_x_DIS, test_window_y_DIS))

# # train_dataset_DIS = train_dataset_DIS.batch(batch, drop_remainder=True)
# # test_dataset_DIS =  test_dataset_DIS.batch(batch, drop_remainder=True)

# for i in range(0, tensor_x_DIS.shape[0]-window_length):
#   window_tensor_x_DIS[i] = tensor_x_DIS[i:i+window_length][:]
#   window_tensor_y_DIS[i] = tensor_y_DIS[i+window_length-1]

# window_tensor_x_DIS = tf.convert_to_tensor(window_tensor_x_DIS)
# window_tensor_y_DIS = tf.convert_to_tensor(window_tensor_y_DIS)

# print(window_tensor_x_DIS.shape)
# print(window_tensor_y_DIS.shape)
# print(test_window_x_DIS.shape)
# print(train_window_x_DIS.shape)